In [1]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,Conv1D,Conv2D,Flatten,BatchNormalization,MaxPooling1D,Dropout,Input,concatenate,GlobalAveragePooling2D,GlobalMaxPooling2D,MaxPooling2D,Activation
import tensorflow as tf
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.utils import resample
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.optimizers import Adam,SGD
import datetime
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard
import os

In [4]:
def load_dataset():
    df = pd.read_csv('./data_test/benign_test_3tr5.csv')
    # df.drop(['Unnamed: 0'],axis=1,inplace=True)
    final = pd.read_csv('../../final_1/final_had_p.csv')
    # drop columns
    df.drop(['Timestamp'], axis=1, inplace=True)
    df.drop(['Protocol','Dst Port'],axis=1,inplace=True)
    
    
    # duplicated
    print(df.duplicated().sum())
    df.drop_duplicates(inplace=True)
    # drop inf
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)

    # drop columns
    print(df.info())

    print(df['Label'].value_counts())
    # drop columns

    train_dataset = df
    print(train_dataset['Label'].value_counts())

    # encode label
    labelE = LabelEncoder()
    train_dataset['Label'] = labelE.fit_transform(train_dataset['Label'])
    train_dataset['Label'].value_counts()
    y = train_dataset['Label']
    train_dataset = train_dataset.drop(['Label'],axis=1)
    final.drop(['Label'],axis=1,inplace=True)

    # normalize data 
    sc = StandardScaler()
    sc.fit(final)
    train_dataset = sc.transform(train_dataset)
    train_dataset = pd.DataFrame(train_dataset,columns=df.columns[:-1])
    
    # split data
    X_train, X_test, y_train, y_test = train_test_split(train_dataset, y, test_size=0.1, random_state=42)
    return X_train, X_test, y_train, y_test

def re_shape(X_train,X_test,y_train,y_test):
    X_train = X_train.to_numpy().reshape(len(X_train),X_train.shape[1],1)
    X_test = X_test.to_numpy().reshape(len(X_test),X_test.shape[1],1)
    return X_train,X_test,y_train,y_test



In [5]:
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model

model = load_model(f'./model_2023-02-28-13-58-10.h5')
X_train, X_test, y_train, y_test = load_dataset()
X_train,X_test,y_train,y_test = re_shape(X_train,X_test,y_train,y_test)

yhat = np.argmax(model.predict([X_train,X_train,X_train]),axis=-1)
print(yhat)
print(y_train)
y_t = np.full(y_train.shape,0)
print(yhat)
# print(len(yhat))
print(accuracy_score(yhat,y_t))
# 2 0.9999935920233507 >350795 password_attack
# 1 0.9986539980975279 >1930829 DDOS
# 0 1.0  350794 and 2826122  normal
# 

803236
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2696113 entries, 0 to 3500793
Data columns (total 77 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Flow Duration      int64  
 1   Tot Fwd Pkts       int64  
 2   Tot Bwd Pkts       int64  
 3   TotLen Fwd Pkts    float64
 4   TotLen Bwd Pkts    float64
 5   Fwd Pkt Len Max    float64
 6   Fwd Pkt Len Min    float64
 7   Fwd Pkt Len Mean   float64
 8   Fwd Pkt Len Std    float64
 9   Bwd Pkt Len Max    float64
 10  Bwd Pkt Len Min    float64
 11  Bwd Pkt Len Mean   float64
 12  Bwd Pkt Len Std    float64
 13  Flow Byts/s        float64
 14  Flow Pkts/s        float64
 15  Flow IAT Mean      float64
 16  Flow IAT Std       float64
 17  Flow IAT Max       float64
 18  Flow IAT Min       float64
 19  Fwd IAT Tot        float64
 20  Fwd IAT Mean       float64
 21  Fwd IAT Std        float64
 22  Fwd IAT Max        float64
 23  Fwd IAT Min        float64
 24  Bwd IAT Tot        float64
 25  Bwd IAT Mea

In [2]:
df = pd.read_csv('../csv_all/all_benign/data_concat_Benign.csv')

In [3]:
# split large file
df = df[:3500795]
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.to_csv('./data_test/benign_test_3tr5.csv',index=False)